### **01 - Incidence of MLTC**
#### **01D02 - Manuscript outputs - age standardisation outputs**

**Imports**

In [1]:
%%pyspark
# required imports
from scipy.stats import norm

# requires blank line after last import


In [2]:
if (!requireNamespace("PHEindicatormethods", quietly = TRUE)) {
  install.packages("PHEindicatormethods")
}
library(PHEindicatormethods)

**Parameter cell**

In [3]:
%%pyspark
# parameter cell
incidence_schema = ""  # "mltc_incidence_outputs_v40_20230331"

# optional, can be blank


In [4]:
%%pyspark
# Set parameters in Spark configuration with 'param.' prefix (for use in SQL cells)
spark.conf.set("param.incidence_schema", incidence_schema)


---

#### **01D02 - Manuscript outputs - age standardisation outputs**

This section uses the phe_dsr R function from the PHEindicatormethods package to apply direct age standardisation, using the input table from the previous step.

**Type of standardisation**: direct standardisation using the national population with 0 conditions as the standard population.

**a - Load data to a Spark DataFrame**

In [6]:
data <- sql("SELECT * FROM ${param.incidence_schema}.mm_incidence_age_standardisation_input")


**b - Convert to an R DataFrame**

In [7]:
r_data <- collect(data)

**c - Create separate R DataFrames for each unique combination of breakdowns**

Create separate R DataFrames for each unique combination of 
- `gender_description`
- `breakdown_type`
- `socio_demographic_breakdown`
- `initial_condition_count`

This is required as the **phe_dsr** function does not accept breakdown columns, instead just calculating the standardised rate across all rows of data.

Each DataFrame will contain the 10 year age band breakdown for the selected combination of breakdowns.

In [8]:
# Step 1: Get unique values for each of the columns
unique_gender <- unique(r_data$gender_description)
unique_breakdown_type <- unique(r_data$breakdown_type)
unique_socio_demographic_breakdown <- unique(r_data$socio_demographic_breakdown)
unique_initial_condition_count <- unique(r_data$initial_condition_count)

# Step 2: Create all possible combinations of these unique values
combinations <- expand.grid(gender_description = unique_gender,
                            breakdown_type = unique_breakdown_type,
                            socio_demographic_breakdown = unique_socio_demographic_breakdown,
                            initial_condition_count = unique_initial_condition_count)

# Step 3: Loop through each combination and create a subset dataframe
dataframes_list <- list() # Initialize an empty list to store dataframes

for(i in 1:nrow(combinations)) {
  # Subset the original dataframe based on the current combination
  subset_r_data <- r_data[r_data$gender_description == combinations$gender_description[i] &
                  r_data$breakdown_type == combinations$breakdown_type[i] &
                  r_data$socio_demographic_breakdown == combinations$socio_demographic_breakdown[i] &
                  r_data$initial_condition_count == combinations$initial_condition_count[i], ]
  
  # Check if the subset dataframe is not empty
  if(nrow(subset_r_data) > 0) {
    # Store the subset dataframe in the list
    r_data_name <- paste("r_data", i, sep = "_")
    dataframes_list[[r_data_name]] <- subset_r_data
  }
}

**d - Apply phe_dsr to each unique combination of breakdowns**

Apply phe_dsr iteratively to each DataFrame created in the step above, with a unique combination of:
- `gender_description`
- `breakdown_type`
- `socio_demographic_breakdown`
- `initial_condition_count`

Insert results into a combined DataFrame for each breakdown.

<blockquote style="color: #333333; background-color: #FFBF00; padding: 10px; border-left: 6px solid #C48800;">
  <strong>⚠️ Warning:</strong> <code>phe_dsr</code> has been soft deprecated in 
  <a href="https://cran.r-project.org/web/packages/PHEindicatormethods/news/news.html">PHEindicatormethods</a>, 
  replaced by <code>calculate_dsr</code>, which has more functionality. This will be removed from the package from September 2025 so should be replaced in any future code.  
</blockquote>



In [9]:
# Initialize an empty dataframe to store the results
results_df <- data.frame()

# Iterate through each subset dataframe in dataframes_list
for(i in seq_along(dataframes_list)) {
  # Extract the current dataframe
  current_df <- dataframes_list[[i]]
  
  # Run phe_dsr on the current dataframe
  dsr_result <- phe_dsr(data = current_df,
                        x = progression_incidence,
                        n = person_years,
                        stdpop = standard_pop_person_years,
                        stdpoptype = "field",
                        confidence = 0.95,
                        multiplier = 100
                        )
  
  # Add the breakdown columns based on values in first row
  if(nrow(current_df) > 0) {
    num_rows <- nrow(dsr_result)
    dsr_result$gender_description <- rep(current_df$gender_description[1], num_rows)
    dsr_result$breakdown_type <- rep(current_df$breakdown_type[1], num_rows)
    dsr_result$socio_demographic_breakdown <- rep(current_df$socio_demographic_breakdown[1], num_rows)
    dsr_result$initial_condition_count <- rep(current_df$initial_condition_count[1], num_rows)
  }
  
  # Bind dsr_result to the results_df
  results_df <- rbind(results_df, dsr_result)
}


**e - Convert back to a Spark DataFrame, then a SQL temporary view**

In [10]:
spark_df <- createDataFrame(results_df)

In [11]:
createOrReplaceTempView(spark_df, "age_standardised_rates")

**f - Calculating age standardised PRRs with confidence intervals**

Progression Rate Ratio (PRR) and Confidence Intervals Calculation:

This SQL section computes the Progression Rate Ratio (PRR) between two populations and their confidence intervals. The PRR is derived from the Directly Standardised Rates (DSR) of each population. 

Key Steps:
- **Standardisation**: Adjust population weights to sum to one.
- **DSR Calculation**: Calculate the age-standardised incidence rate for each population.
- **Variance Calculation**: Determine the variance of each DSR.
- **PRR and Confidence Intervals**: Compute the PRR and its confidence intervals using the delta method and a specified confidence level.

This process allows for a comparison of incidence rates between two populations while accounting for differences in age distribution.

Restructure data into required format

In [13]:
%%sql

CREATE    OR REPLACE TEMPORARY VIEW prr_calculation_input AS
SELECT    u0.gender_description,
          u0.breakdown_type,
          u0.socio_demographic_breakdown,
          u1.initial_condition_count,
          u0.age_band as age,
          u0.person_years AS person_years_population_1,
          u0.progression_incidence AS number_of_events_population_1,
          u1.person_years AS person_years_population_2,
          u1.progression_incidence AS number_of_events_population_2,
          u0.standard_pop_person_years as standard_population_weight
FROM      ${param.incidence_schema}.mm_incidence_age_standardisation_input u0
INNER     JOIN ${param.incidence_schema}.mm_incidence_age_standardisation_input u1 ON u1.gender_description = u0.gender_description
AND       u1.breakdown_type = u0.breakdown_type
AND       u1.socio_demographic_breakdown = u0.socio_demographic_breakdown
AND       u1.age_band = u0.age_band
AND       u1.initial_condition_count = 1
WHERE     u0.initial_condition_count = 0
UNION ALL
SELECT    u0.gender_description,
          u0.breakdown_type,
          u0.socio_demographic_breakdown,
          u2.initial_condition_count,
          u0.age_band as age,
          u0.person_years AS person_years_population_1,
          u0.progression_incidence AS number_of_events_population_1,
          u2.person_years AS person_years_population_2,
          u2.progression_incidence AS number_of_events_population_2,
          u0.standard_pop_person_years as standard_population_weight
FROM      ${param.incidence_schema}.mm_incidence_age_standardisation_input u0
INNER     JOIN ${param.incidence_schema}.mm_incidence_age_standardisation_input u2 ON u2.gender_description = u0.gender_description
AND       u2.breakdown_type = u0.breakdown_type
AND       u2.socio_demographic_breakdown = u0.socio_demographic_breakdown
AND       u2.age_band = u0.age_band
AND       u2.initial_condition_count = 2
WHERE     u0.initial_condition_count = 0

Set confidence level and calculate Z value (and set as a variable to be available in subsequent SQL cells)

In [14]:
%%pyspark
# Define the confidence level
confidence_level = 0.95

# Calculate the Z-value for the given confidence level
z_value = norm.ppf((1 + confidence_level) / 2)

spark.conf.set("param.z_value", str(z_value))


In [15]:
%%sql

CREATE    OR REPLACE TEMPORARY VIEW standardised_weights AS
SELECT    gender_description,
          breakdown_type,
          socio_demographic_breakdown,
          initial_condition_count,
          age,
          person_years_population_1,
          number_of_events_population_1,
          person_years_population_2,
          number_of_events_population_2,
          standard_population_weight / SUM(standard_population_weight) OVER (
          PARTITION BY gender_description,
                    breakdown_type,
                    socio_demographic_breakdown,
                    initial_condition_count
          ) AS standardized_weight
FROM      prr_calculation_input

In [16]:
%%sql

CREATE    OR REPLACE TEMPORARY VIEW dsr_calculation AS
SELECT    gender_description,
          breakdown_type,
          socio_demographic_breakdown,
          initial_condition_count,
          SUM(
          standardized_weight * (number_of_events_population_1 / person_years_population_1)
          ) AS dsr_population_1,
          SUM(
          standardized_weight * (number_of_events_population_2 / person_years_population_2)
          ) AS dsr_population_2,
          SUM(
          (standardized_weight / person_years_population_1) * (standardized_weight / person_years_population_1) * number_of_events_population_1
          ) AS var_pop1,
          SUM(
          (standardized_weight / person_years_population_2) * (standardized_weight / person_years_population_2) * number_of_events_population_2
          ) AS var_pop2
FROM      standardised_weights
GROUP BY  gender_description,
          breakdown_type,
          socio_demographic_breakdown,
          initial_condition_count

In [17]:
%%sql

CREATE    OR REPLACE TEMPORARY VIEW prr_calculation AS
SELECT    gender_description,
          breakdown_type,
          socio_demographic_breakdown,
          initial_condition_count,
          dsr_population_1,
          dsr_population_2,
          dsr_population_2 / dsr_population_1 AS prr,
          var_pop2 / (dsr_population_2 * dsr_population_2) + var_pop1 / (dsr_population_1 * dsr_population_1) AS var_log_prr
FROM      dsr_calculation

In [18]:
%%sql

CREATE    OR REPLACE TEMPORARY VIEW confidence_intervals AS
SELECT    *,
          EXP(LOG(prr) - ${param.z_value} * SQRT(var_log_prr)) AS ci_lower_prr,
          EXP(LOG(prr) + ${param.z_value} * SQRT(var_log_prr)) AS ci_upper_prr
FROM      prr_calculation

In [19]:
%%sql

CREATE    OR REPLACE TEMPORARY VIEW age_standardised_prr AS
SELECT    gender_description,
          breakdown_type,
          socio_demographic_breakdown,
          initial_condition_count,
          dsr_population_1,
          dsr_population_2,
          prr,
          ci_lower_prr,
          ci_upper_prr
FROM      confidence_intervals

**Extract results**

This first query extracts the main results
- The combined IMD and ethnicity output is excluded, as it is dealt with separately below
- Small number suppression is applied to the standard population and expected incidence figures (although is unlikely to be required, given these volumes are based on the standard population of people with 0 conditions)

<blockquote style="color: #D8000C; background-color: #FFD2D2; padding: 10px; border-left: 6px solid #D8000C;">
  <strong>⚠️ Warning:</strong> DROP TABLE is currently commented out, as this table does not need to be recreated each time the incidence analysis is run.
</blockquote>

In [20]:
%%sql

--DROP TABLE IF EXISTS ${param.incidence_schema}.output_01D02_incidence_results_age_standardisation

In [21]:
%%sql

CREATE    TABLE ${param.incidence_schema}.output_01D02_incidence_results_age_standardisation USING PARQUET AS
SELECT    a0.gender_description,
          a0.breakdown_type,
          a0.socio_demographic_breakdown,
          CASE WHEN a0.total_pop BETWEEN 1 AND 7 THEN '***' ELSE CAST(a0.total_pop AS STRING) END AS sp_person_years_0,
          CASE WHEN a0.total_count BETWEEN 1 AND 7 THEN '***' ELSE CAST(a0.total_count AS STRING) END AS exp_incidence_0_1_plus,
          a0.value AS progression_rate_0_1_plus,
          a0.lowercl AS lower_cl_0_1,
          a0.uppercl AS upper_cl_0_1,
          CASE WHEN a1.total_pop BETWEEN 1 AND 7 THEN '***' ELSE CAST(a1.total_pop AS STRING) END AS sp_person_years_1,
          CASE WHEN a1.total_count BETWEEN 1 AND 7 THEN '***' ELSE CAST(a1.total_count AS STRING) END AS exp_incidence_1_2_plus,
          a1.value AS progression_rate_1_2_plus,
          a1.lowercl AS lower_cl_1_2,
          a1.uppercl AS upper_cl_1_2,
          prr1.prr AS prr_1_2,
          prr1.ci_lower_prr AS lower_cl_prr_1_2,
          prr1.ci_upper_prr AS upper_cl_prr_1_2,
          CASE WHEN a2.total_pop BETWEEN 1 AND 7 THEN '***' ELSE CAST(a2.total_pop AS STRING) END AS sp_person_years_2,
          CASE WHEN a2.total_count BETWEEN 1 AND 7 THEN '***' ELSE CAST(a2.total_count AS STRING) END AS exp_incidence_2_3_plus,
          a2.value AS progression_rate_2_3_plus,
          a2.lowercl AS lower_cl_2_3,
          a2.uppercl AS upper_cl_2_3,
          prr2.prr AS prr_2_3,
          prr2.ci_lower_prr AS lower_cl_prr_2_3,
          prr2.ci_upper_prr AS upper_cl_prr_2_3
FROM      age_standardised_rates a0
INNER     JOIN age_standardised_rates a1 ON a1.gender_description = a0.gender_description
AND       a1.breakdown_type = a0.breakdown_type
AND       a1.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       a1.initial_condition_count = 1
INNER     JOIN age_standardised_rates a2 ON a2.gender_description = a0.gender_description
AND       a2.breakdown_type = a0.breakdown_type
AND       a2.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       a2.initial_condition_count = 2
INNER     JOIN age_standardised_prr prr1 ON prr1.gender_description = a0.gender_description
AND       prr1.breakdown_type = a0.breakdown_type
AND       prr1.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       prr1.initial_condition_count = 1
INNER     JOIN age_standardised_prr prr2 ON prr2.gender_description = a0.gender_description
AND       prr2.breakdown_type = a0.breakdown_type
AND       prr2.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       prr2.initial_condition_count = 2
WHERE     a0.initial_condition_count = 0
AND       a0.breakdown_type <> 'IMD and Ethnicity'
AND       a0.breakdown_type <> 'Gender, IMD and Ethnicity'
ORDER BY  CASE
                    WHEN a0.gender_description = 'NA' THEN 0
                    ELSE a0.gender_description
          END,
          CASE
                    WHEN a0.breakdown_type = 'NA' THEN 0
                    ELSE a0.breakdown_type
          END,
          CASE
                    WHEN a0.socio_demographic_breakdown = 'NA' THEN 0
                    ELSE a0.socio_demographic_breakdown
          END,
          a0.initial_condition_count

In [22]:
%%sql

SELECT    *
FROM      ${param.incidence_schema}.output_01D02_incidence_results_age_standardisation

This second query extracts results for the combined IMD and ethnicity output.
- This breakdown is dealt with separately, as IMD and ethnicity values are combined within the same `socio_demographic_breakdown` column, separated by a " / " delimiter
- `socio_demographic_breakdown` is therefore split into two separate columns for IMD and Ethnicity
- Small number suppression is applied to the standard population and expected incidence figures (although is unlikely to be required, given these volumes are based on the standard population of people with 0 conditions)

<blockquote style="color: #D8000C; background-color: #FFD2D2; padding: 10px; border-left: 6px solid #D8000C;">
  <strong>⚠️ Warning:</strong> DROP TABLE is currently commented out, as this table does not need to be recreated each time the incidence analysis is run.
</blockquote>

In [ ]:
%%sql

--DROP TABLE IF EXISTS ${param.incidence_schema}.output_01D02_incidence_results_age_standardisation_ethnicity_imd

In [23]:
%%sql

CREATE    TABLE ${param.incidence_schema}.output_01D02_incidence_results_age_standardisation_ethnicity_imd USING PARQUET AS
SELECT    a0.gender_description,
          a0.breakdown_type,
          split(a0.socio_demographic_breakdown, ' / ')[0] AS IMD, 
          split(a0.socio_demographic_breakdown, ' / ')[1] AS ethnicity,
          CASE WHEN a0.total_pop BETWEEN 1 AND 7 THEN '***' ELSE CAST(a0.total_pop AS STRING) END AS sp_person_years_0,
          CASE WHEN a0.total_count BETWEEN 1 AND 7 THEN '***' ELSE CAST(a0.total_count AS STRING) END AS exp_incidence_0_1_plus,
          a0.value AS progression_rate_0_1_plus,
          a0.lowercl AS lower_cl_0_1,
          a0.uppercl AS upper_cl_0_1,
          CASE WHEN a1.total_pop BETWEEN 1 AND 7 THEN '***' ELSE CAST(a1.total_pop AS STRING) END AS sp_person_years_1,
          CASE WHEN a1.total_count BETWEEN 1 AND 7 THEN '***' ELSE CAST(a1.total_count AS STRING) END AS exp_incidence_1_2_plus,
          a1.value AS progression_rate_1_2_plus,
          a1.lowercl AS lower_cl_1_2,
          a1.uppercl AS upper_cl_1_2,
          prr1.prr AS prr_1_2,
          prr1.ci_lower_prr AS lower_cl_prr_1_2,
          prr1.ci_upper_prr AS upper_cl_prr_1_2,
          CASE WHEN a2.total_pop BETWEEN 1 AND 7 THEN '***' ELSE CAST(a2.total_pop AS STRING) END AS sp_person_years_2,
          CASE WHEN a2.total_count BETWEEN 1 AND 7 THEN '***' ELSE CAST(a2.total_count AS STRING) END AS exp_incidence_2_3_plus,
          a2.value AS progression_rate_2_3_plus,
          a2.lowercl AS lower_cl_2_3,
          a2.uppercl AS upper_cl_2_3,
          prr2.prr AS prr_2_3,
          prr2.ci_lower_prr AS lower_cl_prr_2_3,
          prr2.ci_upper_prr AS upper_cl_prr_2_3
FROM      age_standardised_rates a0
INNER     JOIN age_standardised_rates a1 ON a1.gender_description = a0.gender_description
AND       a1.breakdown_type = a0.breakdown_type
AND       a1.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       a1.initial_condition_count = 1
INNER     JOIN age_standardised_rates a2 ON a2.gender_description = a0.gender_description
AND       a2.breakdown_type = a0.breakdown_type
AND       a2.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       a2.initial_condition_count = 2
INNER     JOIN age_standardised_prr prr1 ON prr1.gender_description = a0.gender_description
AND       prr1.breakdown_type = a0.breakdown_type
AND       prr1.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       prr1.initial_condition_count = 1
INNER     JOIN age_standardised_prr prr2 ON prr2.gender_description = a0.gender_description
AND       prr2.breakdown_type = a0.breakdown_type
AND       prr2.socio_demographic_breakdown = a0.socio_demographic_breakdown
AND       prr2.initial_condition_count = 2
WHERE     a0.initial_condition_count = 0
AND       (
            a0.breakdown_type = 'IMD and Ethnicity'
OR          a0.breakdown_type = 'Gender, IMD and Ethnicity'
          )
ORDER BY  CASE
                    WHEN a0.gender_description = 'NA' THEN 0
                    ELSE a0.gender_description
          END,
          a0.breakdown_type,
          split(a0.socio_demographic_breakdown, ' / ')[0], 
          split(a0.socio_demographic_breakdown, ' / ')[1],
          a0.initial_condition_count

In [24]:
%%sql

SELECT    *
FROM      ${param.incidence_schema}.output_01D02_incidence_results_age_standardisation_ethnicity_imd